# Обучение модели классификации комментариев

### Описание проекта

Интернет-магазин запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Требуется инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

### План проекта

    1. Загрузка и подготовка данных
    2. Обучение моделей и тестирование лучшей
    3. Выводы

### Описание данных

В распоряжении набор данных с разметкой о токсичности правок.

## 1. Загрузка и подготовка данных

In [66]:
# загужаем необходимые библиотеки
import pandas as pd
import numpy as np
import re
import random
import time
import nltk

!pip install lightgbm


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score
from nltk.tokenize import word_tokenize  
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from lightgbm import LGBMClassifier

nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Пользователь\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Пользователь\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Пользователь\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [7]:
# для воспроизводимости результатов
random.seed(42)

In [3]:
# загружаем данные
try:
    data = pd.read_csv('toxic_comments.csv', index_col=0)
except:
    data = pd.read_csv('/datasets/toxic_comments.csv', index_col=0)

In [5]:
# выведем общую информацию:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB


In [9]:
# выведем 10 случайных строк для ознакомления:
data.sample(10)

,text,toxic
44375,"""With all of the TALK / """"lies"""" that Chappell...",0
56397,"Yeah I'm pretty much of a newbie here, actuall...",0
152370,I wonder if WikiProject The Isles? might work?,0
17221,ok i'll come back. But i get annoyed when ppl ...,0
44097,mill dickwad admin]] = Hilarity ensues by bloc...,0
61031,""":::I'm sure the Nazi scientists who performed...",0
54922,ha ha how are you gonna stop me?,0
91901,numerous evidence on the issue \n\nThe Himmler...,0
80693,"""\n\n Vad gör du? \n\nVarför raderar du text s...",0
105996,I Could Sing Of Your Love Forever \n\nI Could ...,0


In [11]:
# проверка на сбалансированность классов:
data['toxic'].value_counts()

0    143106
1     16186
Name: toxic, dtype: int64

In [12]:
#создаем функцию для регуляризации текстов:
def texts_re(text):
    text_re = re.sub(r'[^a-zA-Z\']', ' ', text)
    text_list = text_re.split()
    return ' '.join(text_list)

In [13]:
# регуляризация исходных данных
corpus_re = data['text'].apply(lambda x: texts_re(x))

In [18]:
%%time
# токенизация исходных данных:
#stemmer = PorterStemmer()
tokens = corpus_re.apply(lambda x: word_tokenize(x))

CPU times: total: 1min 1s
Wall time: 1min 1s


In [26]:
# функция для определения POS-тегов слов:
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

In [23]:
# создаем объект WordNetLemmatizer для лемматизации:
lemmatizer = WordNetLemmatizer()

In [29]:
# функция для лемматизации текстов:
def lemmatize_str(tokens_str):
    lemmatize_tokens_str = []
    for word in tokens_str:
        if len(word)<35:
            lemmatize_tokens_str.append(lemmatizer.lemmatize(word, get_wordnet_pos(word)))
    return lemmatize_tokens_str

In [32]:
%%time
# лемматизация исходных данных
lemmatize_tokens = tokens.apply(lambda x: lemmatize_str(x))

CPU times: total: 1h 48min 23s
Wall time: 1h 51min 13s


In [36]:
# получаем корпус лемматизированных текстов
corpus = lemmatize_tokens.apply(lambda x: " ".join(x))

In [37]:
# приводим к нижнему регистру:
corpus = corpus.apply(lambda x: x.lower())

In [38]:
# выведем случайные 10 строк для просмотра результата очистки и лемматизации:
corpus.sample(10)

54090     the equation use in multidimensional newton ra...
74813     pointing to a list that include blog and the w...
52568     march utc it appear ron that you be frustratin...
18614     threat the comment be delete for the reason i ...
104865    hi there 'fruiters do you think file alfred ja...
140344                   acceptable a far a anyone can know
157703             it 's all explain very clearly at wp say
32403     greek club stats somebody please provide a rel...
1518      ripieno concerto a cinque two viola score one ...
130778    followup ten edits from this ip to date includ...
Name: text, dtype: object

In [41]:
# разделяем данные на обучающую и тестовую выборки:
features_train, features_test, target_train, target_test = train_test_split(corpus, data['toxic'], test_size=0.25, \
                                                                            stratify=data['toxic'])

In [42]:
# загружаем англоязычные стоп-слова
stop_words = set(stopwords.words('english'))

## 2. Обучение моделей и тестирование лучшей

**1 Модель логистической регрессии:**

In [56]:
%%time
# параметры логистической регрессии:
params={'tfidf__ngram_range':[(1,1), (1,2), (1,4)], \
        'model__C':[.1,.5,1,5,10]} 
# создание pipeline для вычисления TF-IDF и модели логистической регрессии:
pipe = Pipeline([ 
    ('tfidf', TfidfVectorizer(stop_words=stop_words)), 
    ('model',LogisticRegression())]) 
grid_linear = RandomizedSearchCV(pipe, params, cv=3, scoring='f1')
grid_linear.fit(features_train, target_train)

D:\Anaconda\envs\ds_practicum_env\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
D:\Anaconda\envs\ds_practicum_env\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-re

CPU times: total: 22min 3s
Wall time: 21min 45s


D:\Anaconda\envs\ds_practicum_env\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


RandomizedSearchCV(cv=3,
                   estimator=Pipeline(steps=[('tfidf',
                                              TfidfVectorizer(stop_words={'a',
                                                                          'about',
                                                                          'above',
                                                                          'after',
                                                                          'again',
                                                                          'against',
                                                                          'ain',
                                                                          'all',
                                                                          'am',
                                                                          'an',
                                                                          'and',
                     

**2 Модель LGBMClassifier:**

In [59]:
%%time
# параметры TF-IDF и LGBMClassifier:
params={'tfidf__ngram_range':[(1,1), (1,4)], \
        'model__max_depth': [15, 20, 25, 30], \
        'model__learning_rate':[.01,.05,.1,.15]} 
# создание pipeline для вычисления TF-IDF и модели LGBMClassifier:
pipe = Pipeline([ 
    ('tfidf', TfidfVectorizer(stop_words=stop_words)), 
    ('model',LGBMClassifier())]) 
grid_lgbm = RandomizedSearchCV(pipe, params, cv=3, scoring='f1')
grid_lgbm.fit(features_train, target_train)

RandomizedSearchCV(cv=3,
                   estimator=Pipeline(steps=[('tfidf',
                                              TfidfVectorizer(stop_words={'a',
                                                                          'about',
                                                                          'above',
                                                                          'after',
                                                                          'again',
                                                                          'against',
                                                                          'ain',
                                                                          'all',
                                                                          'am',
                                                                          'an',
                                                                          'and',
                     

In [62]:
# таблица с полученными оценками обученных моделей:
criteria = pd.DataFrame(0, columns=['f1_score'], \
                        index=['LogisticRegression', 'LGBMClassifier'])
criteria.loc['LogisticRegression', 'f1_score'] = round(grid_linear.best_score_, 2)
criteria.loc['LGBMClassifier', 'f1_score'] = round(grid_lgbm.best_score_, 2)
criteria

,f1_score
LogisticRegression,0.77
LGBMClassifier,0.76


In [65]:
# расчет f1 лучшей модели на тестовой выборке:
predicted_test = grid_linear.best_estimator_.predict(features_test)
f1_score(target_test, predicted_test)

0.772075055187638

## 3. Выводы

    1. В результате подготовки исходных данных:
        - данные быди загружены, всего 159292 текстов
        - произвели регуляризацию исходных текстов
        - произвели токенизацию и лемматиацию исходных текстов
        - разделили исходные данные на обучающую и тестовую выборки
        - обучили счетчик для расчета TF-IDF на обучающей выборке
        - произвели расчет TF-IDF для обучающей выборки
    2. В результате обучения моделей:
        - подобрали гиперпараметры для модели логистической регрессии
        - подобрали гиперпараметры для модели случайного леса
        - подобрали гиперпараметры для модели LGBMClassifier
        - моделью с наилучшим значением метрики f1 является LGBMClassifier
        - произвели расчет TF-IDF для тестовой выборки выборки
        - произвели расчет f1 модели LGBMClassifier на тестовой выборке (f1=0.77)
        - заданная точность достигнута